# Day 16 - claude

In [1]:
from collections import defaultdict
import heapq
from typing import List, Tuple, Dict, Set

def read_maze(filename: str) -> List[str]:
    """Read the maze from input file."""
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]

class Direction:
    EAST = (0, 1)
    WEST = (0, -1)
    NORTH = (-1, 0)
    SOUTH = (1, 0)

    @staticmethod
    def get_rotations(current: Tuple[int, int]) -> List[Tuple[int, int]]:
        """Get possible rotations (90 degrees clockwise and counterclockwise)."""
        directions = [Direction.NORTH, Direction.EAST, Direction.SOUTH, Direction.WEST]
        current_idx = directions.index(current)
        # Clockwise and counterclockwise rotations
        return [
            directions[(current_idx + 1) % 4],  # Clockwise
            directions[(current_idx - 1) % 4]   # Counterclockwise
        ]

def find_start_end(maze: List[str]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """Find start (S) and end (E) positions in the maze."""
    start = end = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'S':
                start = (i, j)
            elif cell == 'E':
                end = (i, j)
    return start, end

def solve_maze(maze: List[str]) -> int:
    """Find the lowest possible score to reach the end."""
    start_pos, end_pos = find_start_end(maze)
    
    # State: (position, direction)
    # Initially facing east as per problem statement
    initial_state = (start_pos, Direction.EAST)
    
    # Priority queue: (score, position, direction)
    queue = [(0, start_pos, Direction.EAST)]
    # Keep track of best scores for each state
    best_scores: Dict[Tuple[Tuple[int, int], Tuple[int, int]], int] = defaultdict(lambda: float('inf'))
    best_scores[initial_state] = 0
    
    while queue:
        score, pos, direction = heapq.heappop(queue)
        
        # Skip if we've found a better path to this state
        if score > best_scores[(pos, direction)]:
            continue
        
        # Check if we've reached the end
        if pos == end_pos:
            return score
        
        # Try rotations (cost: 1000 each)
        for new_direction in Direction.get_rotations(direction):
            new_score = score + 1000
            new_state = (pos, new_direction)
            
            if new_score < best_scores[new_state]:
                best_scores[new_state] = new_score
                heapq.heappush(queue, (new_score, pos, new_direction))
        
        # Try moving forward (cost: 1)
        new_row = pos[0] + direction[0]
        new_col = pos[1] + direction[1]
        new_pos = (new_row, new_col)
        
        # Check if the move is valid (within bounds and not a wall)
        if (0 <= new_row < len(maze) and 
            0 <= new_col < len(maze[0]) and 
            maze[new_row][new_col] != '#'):
            
            new_score = score + 1
            new_state = (new_pos, direction)
            
            if new_score < best_scores[new_state]:
                best_scores[new_state] = new_score
                heapq.heappush(queue, (new_score, new_pos, direction))
    
    return float('inf')  # No path found

def main():
    maze = read_maze('input.txt')
    result = solve_maze(maze)
    print(f"Lowest possible score: {result}")

if __name__ == "__main__":
    main()

Lowest possible score: 66404


## Part 2

In [2]:
from collections import defaultdict, deque
import heapq
from typing import List, Tuple, Dict, Set, Optional

def read_maze(filename: str) -> List[str]:
    """Read the maze from input file."""
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]

class Direction:
    EAST = (0, 1)
    WEST = (0, -1)
    NORTH = (-1, 0)
    SOUTH = (1, 0)

    @staticmethod
    def get_rotations(current: Tuple[int, int]) -> List[Tuple[int, int]]:
        """Get possible rotations (90 degrees clockwise and counterclockwise)."""
        directions = [Direction.NORTH, Direction.EAST, Direction.SOUTH, Direction.WEST]
        current_idx = directions.index(current)
        return [
            directions[(current_idx + 1) % 4],  # Clockwise
            directions[(current_idx - 1) % 4]   # Counterclockwise
        ]

def find_start_end(maze: List[str]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """Find start (S) and end (E) positions in the maze."""
    start = end = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'S':
                start = (i, j)
            elif cell == 'E':
                end = (i, j)
    return start, end

def solve_maze(maze: List[str]) -> Tuple[int, Set[Tuple[int, int]]]:
    """Find the lowest possible score and all tiles that are part of optimal paths."""
    start_pos, end_pos = find_start_end(maze)
    
    # State: (position, direction)
    initial_state = (start_pos, Direction.EAST)
    
    # Priority queue: (score, position, direction)
    queue = [(0, start_pos, Direction.EAST)]
    # Keep track of best scores for each state
    best_scores: Dict[Tuple[Tuple[int, int], Tuple[int, int]], int] = defaultdict(lambda: float('inf'))
    best_scores[initial_state] = 0
    
    # Keep track of predecessors for path reconstruction
    predecessors: Dict[Tuple[Tuple[int, int], Tuple[int, int]], List[Tuple[Tuple[int, int], Tuple[int, int]]]] = defaultdict(list)
    
    optimal_score = float('inf')
    end_states = []  # List of all states that reach the end with optimal score
    
    while queue:
        score, pos, direction = heapq.heappop(queue)
        
        # Skip if we've found a better path to this state
        current_state = (pos, direction)
        if score > best_scores[current_state]:
            continue
        
        # If we've reached the end, update optimal score and record the state
        if pos == end_pos:
            if score < optimal_score:
                optimal_score = score
                end_states = [current_state]
            elif score == optimal_score:
                end_states.append(current_state)
            continue
        
        # Skip exploring further if we're already worse than known optimal solution
        if score >= optimal_score:
            continue
        
        # Try rotations (cost: 1000 each)
        for new_direction in Direction.get_rotations(direction):
            new_score = score + 1000
            new_state = (pos, new_direction)
            
            if new_score <= best_scores[new_state]:
                if new_score < best_scores[new_state]:
                    predecessors[new_state] = []
                predecessors[new_state].append(current_state)
                best_scores[new_state] = new_score
                heapq.heappush(queue, (new_score, pos, new_direction))
        
        # Try moving forward (cost: 1)
        new_row = pos[0] + direction[0]
        new_col = pos[1] + direction[1]
        new_pos = (new_row, new_col)
        
        # Check if the move is valid
        if (0 <= new_row < len(maze) and 
            0 <= new_col < len(maze[0]) and 
            maze[new_row][new_col] != '#'):
            
            new_score = score + 1
            new_state = (new_pos, direction)
            
            if new_score <= best_scores[new_state]:
                if new_score < best_scores[new_state]:
                    predecessors[new_state] = []
                predecessors[new_state].append(current_state)
                best_scores[new_state] = new_score
                heapq.heappush(queue, (new_score, new_pos, direction))
    
    # Collect all positions that are part of optimal paths using BFS
    optimal_positions = set()
    queue = deque(end_states)
    visited_states = set()
    
    while queue:
        current_state = queue.popleft()
        if current_state in visited_states:
            continue
            
        visited_states.add(current_state)
        pos, _ = current_state
        optimal_positions.add(pos)
        
        # Add all predecessors to queue
        for pred_state in predecessors[current_state]:
            queue.append(pred_state)
    
    return optimal_score, optimal_positions

def visualize_paths(maze: List[str], optimal_positions: Set[Tuple[int, int]]) -> List[str]:
    """Create a visualization of the maze with optimal paths marked."""
    result = []
    for i, row in enumerate(maze):
        new_row = ''
        for j, cell in enumerate(row):
            if cell != '#' and (i, j) in optimal_positions:
                new_row += 'O'
            else:
                new_row += cell
        result.append(new_row)
    return result

def main():
    maze = read_maze('input.txt')
    optimal_score, optimal_positions = solve_maze(maze)
    
    print(f"Lowest possible score: {optimal_score}")
    print(f"Number of tiles in optimal paths: {len(optimal_positions)}")
    
    # Print visualization
    print("\nVisualization of optimal paths:")
    visualization = visualize_paths(maze, optimal_positions)
    for row in visualization:
        print(row)

if __name__ == "__main__":
    main()

Lowest possible score: 66404
Number of tiles in optimal paths: 433

Visualization of optimal paths:
#############################################################################################################################################
#...........#.........#.#.......#....OOOOOOOOOOOOOOOOOOOOO..#..OOOOOOOOOOO#OOOOOOOOO#........OOOOOOO#OOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOO#
#.#########.#.#######.#.#.#####.#.###O###.#####.#.#.#####O###.#O#########O#O#####.#O#########O#####O#O#####O#####O#.#####O#O###.#.#######.#.#
#.#.......#.#.#...#...#.......#.#...#O#.......#.#.#.#OOOOO#...#O#OOOOOOOOO#O#.....#OOOOOOOOOOO#....OOOOOOOOOOOOOOO#.#...#O#OOO#...#.#.......#
#.#.###.#.#.#.###.#.###########.###.#O#######.#.#.###O#.###.###O#O#########O#.#################.#.###.###.#.#.#.#.#.#.#.#O###O#####.#.#####.#
#.......#.#.#...#.........#.....#...#OOO#...#...#.#OOO#.#...#..O#OOOOOOOOOOO#.....#.........#...#.#.....#.#...#.#.#.#.#.#OOOOO..#...#...#.#.#
#.#####.###.#.#.#########.#.#####.#####O#.#.###.